## Project 2: Digit Recognition

### Setup

1. Note on software: For all the projects, we will use python 3.6 augmented with the NumPy numerical toolbox, the matplotlib plotting toolbox. In this project, we will also use the scikit-learn package, which you could install in the same way you installed other packages, as described in project 0, e.g. by conda install scikit-learn or pip install sklearn

2. Download mnist.tar.gz and untar it into a working directory. The archive contains the various data files in the Dataset directory, along with the following python files:

   - `part1/linear_regression.py` where you will implement linear regression
   - `part1/svm.py` where you will implement support vector machine
   - `part1/softmax.py` where you will implement multinomial regression
   - `part1/features.py` where you will implement principal component analysis (PCA) dimensionality reduction
   - `part1/kernel.py` where you will implement polynomial and Gaussian RBF kernels
   - `part1/main.py` where you will use the code you write for this part of the project

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from mnist import utils as mnist_utils
from typing import Tuple

# Reload custom package
import importlib
importlib.reload(mnist_utils)

# Get MNist data
train_x, train_y, test_x, test_y = mnist_utils.get_MNIST_data()
train_x

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

------

### 2. Linear Regression with Closed Form Solution

#### Closed Form Solution of Linear Regression

$$ \displaystyle  \displaystyle \theta = (X^ T X + \lambda I)^{-1} X^ T Y $$

In [2]:
def closed_form(X, Y, lambda_factor):
    """
    Computes the closed form solution of linear regression with L2 regularization

    Args:
        X - (n, d + 1) NumPy array (n datapoints each with d features plus the bias feature in the first dimension)
        Y - (n, ) NumPy array containing the labels (a number from 0-9) for each
            data point
        lambda_factor - the regularization constant (scalar)
    Returns:
        theta - (d + 1, ) NumPy array containing the weights of linear regression. Note that theta[0]
        represents the y-axis intercept of the model and therefore X[0] = 1
    """
    # Number of training features
    num_features = X.shape[1]

    # Identity matrix
    I = np.identity(num_features)

    # Closed form solution
    theta = np.dot( np.linalg.inv( np.dot(X.T, X) + lambda_factor * I ),  np.dot(X.T, Y) )
    return theta

# Regularization parameter
lambda_factor = 0.01

# Get the models parameters
theta = closed_form(train_x, train_y, lambda_factor)

# Get the predicted value for Y (X * Theta)
test_y_pred = np.round(np.dot(test_x, theta))

# Truncate values to a min of 0 and a max of 9
test_y_pred[test_y_pred < 0] = 0
test_y_pred[test_y_pred > 9] = 9

# Get the percentage of wrong guesses
# (100% - Sum of all the correct guesses divided by the total number of guesses or the mean)
error = 1 - np.mean(test_y_pred == test_y)

print("Error Percentage:", error)

Error Percentage: 0.744


Alice and you find that no matter what $\lambda$ factor you try, the test error is large. With some thinking, you realize that something is wrong with this approach.

- Answer: The loss function related to the closed form solution is inadequate for this problem.

- Reason: The loss function used in regression is least square which does not work well for classification problems. Instead you should use logistic loss function for this task. Suppose you have a binary classification problem, where you want to label data points as +1 or -1. The way you typically set up a classification problem is as follows — for a given data point x, you calculate a function f(x) and if that function value is large enough, you classify that point as +1, otherwise as -1. Now, if you use one of the standard classification losses like hinge loss or logistic loss, then for large values of f(x), if your true label is +1, then these losses go to zero. That is, you do not want to penalize correctly labeled points, and therefore keep them away from interfering with your parameter updates. However, with mean squared error, if your f(x) is anything other than +1, you incur a loss, and even correctly classified points influence the gradients, making learning harder.

------

### 3. Support Vector Machine

#### One vs. Rest SVM

Use the sklearn package and build the SVM model on your local machine. Use random_state = 0, C=0.1 and default values for other parameters.

In [3]:
from sklearn.svm import LinearSVC

def one_vs_rest_svm(train_x, train_y, test_x):
    """
    Trains a linear SVM for binary classifciation

    Args:
        train_x - (n, d) NumPy array (n datapoints each with d features)
        train_y - (n, ) NumPy array containing the labels (0 or 1) for each training data point
        test_x - (m, d) NumPy array (m datapoints each with d features)
    Returns:
        pred_test_y - (m,) NumPy array containing the labels (0 or 1) for each test data point
    """

    # Regularization parameter and the random state used for 
    # the pseudo random number generator inside the SVM
    C = 0.1
    random_state = 0

    # Create the SVM model
    svc = LinearSVC(C = C, random_state = random_state)
    
    # Train the model
    svc.fit(train_x, train_y)

    # Predict new data using the previous training
    pred_test_y = svc.predict(test_x)

    # Get the parameters for this estimator
    params = [None, None]
    params[0] = svc.coef_
    params[1] = svc.intercept_

    return pred_test_y, params


# Now the classifiction will go into detecting if the digit is either not a zero (1)
# or a zero (0). Since the labels in train_y come values from 0 to 9, all the values
# from 1-9 are truncated to 1.
train_y_one_vs_rest = train_y.copy()
train_y_one_vs_rest[train_y_one_vs_rest > 0] = 1

# Predict new data using the test data, while using the training data to generate a model
y_pred, params = one_vs_rest_svm(train_x, train_y_one_vs_rest, test_x)

# Change the classification labels for the test data as well
test_y_one_vs_rest = test_y.copy()
test_y_one_vs_rest[test_y_one_vs_rest > 0] = 1

# Get the error percentage
error = 1 - np.mean(y_pred == test_y_one_vs_rest)
print("Error Percentage:", error)

Error Percentage: 0.007499999999999951


#### Multiclass SVM

In fact, sklearn already implements a multiclass SVM with a one-vs-rest strategy. Use LinearSVC to build a multiclass SVM model

In [4]:
def multi_class_svm(train_x, train_y, test_x):
    """
    Trains a linear SVM for multiclass classifciation using a one-vs-rest strategy

    Args:
        train_x - (n, d) NumPy array (n datapoints each with d features)
        train_y - (n, ) NumPy array containing the labels (int) for each training data point
        test_x - (m, d) NumPy array (m datapoints each with d features)
    Returns:
        pred_test_y - (m,) NumPy array containing the labels (int) for each test data point
    """

    # Regularization parameter and the random state used for 
    # the pseudo random number generator inside the SVM
    C = 0.1
    random_state = 0

    # Create the SVM model
    svc = LinearSVC(C = C, random_state = random_state)
    
    # Train the model
    svc.fit(train_x, train_y)

    # Predict new data using the previous training
    pred_test_y = svc.predict(test_x)
    return pred_test_y


# Predict new data using the test data, while using the training data to generate a model
y_pred = multi_class_svm(train_x, train_y, test_x)

# Get the error percentage
error = 1 - np.mean(y_pred == test_y)
print("Error Percentage:", error)

Error Percentage: 0.08189999999999997


------

### 4. Multinomial (Softmax) Regression and Gradient Descent

#### Computing Probabilities for Softmax

Write a function compute_probabilities that computes, for each data point $x^{(i)}$, the probability that $x^{(i)}$ is labeled as $j$ for $j = 0,1,\dots ,k-1$.

The softmax function $h$ for a particular vector $x$ requires computing

$$h(x) = \frac{1}{\sum _{j=0}^{k-1} e^{\theta _ j \cdot x / \tau }} \begin{bmatrix}  e^{\theta _0 \cdot x / \tau } \\ e^{\theta _1 \cdot x / \tau } \\ \vdots \\ e^{\theta _{k-1} \cdot x / \tau } \end{bmatrix},$$

where $\tau >0$ is the **temperature parameter**. When computing the output probabilities (they should always be in the range $[0,1]$), the terms $e^{\theta _ j \cdot x / \tau }$ may be very large or very small, due to the use of the exponential function. This can cause numerical or overflow errors. To deal with this, we can simply subtract some fixed amount  from each exponent to keep the resulting number from getting too large. Since

$$h(x) = \frac{e^{-c}}{e^{-c}\sum _{j=0}^{k-1} e^{\theta _ j \cdot x / \tau }} \begin{bmatrix}  e^{\theta _0 \cdot x / \tau } \\ e^{\theta _1 \cdot x / \tau } \\ \vdots \\ e^{\theta _{k-1} \cdot x / \tau } \end{bmatrix} \\ = \frac{1}{\sum _{j=0}^{k-1} e^{[\theta _ j \cdot x / \tau ] - c}} \begin{bmatrix}  e^{[\theta _0 \cdot x / \tau ] - c} \\ e^{[\theta _1 \cdot x / \tau ] - c} \\ \vdots \\ e^{[\theta _{k-1} \cdot x / \tau ] - c} \end{bmatrix}$$
 
subtracting some fixed amount $c$ from each exponent will not change the final probabilities. A suitable choice for this fixed amount is $c = \max _ j \theta _ j \cdot x / \tau$.

In [5]:
def compute_probabilities(X, theta, temp_parameter):
    """
    Computes, for each datapoint X[i], the probability that X[i] is labeled as j
    for j = 0, 1, ..., k-1

    Args:
        X - (n, d) NumPy array (n datapoints each with d features)
        theta - (k, d) NumPy array, where row j represents the parameters of our model for label j
        temp_parameter - the temperature parameter of softmax function (scalar)
    Returns:
        H - (k, n) NumPy array, where each entry H[j][i] is the probability that X[i] is labeled as j
    """

    # Predict the "y's" from the dataset using X and theta
    Y = np.dot(theta, X.T)

    # Apply the temperature parameter to the predictions
    Y_temp = Y / temp_parameter

    # Get the max value for the "Y_temp" exponent
    c = np.max(Y_temp, axis=0)

    # Get the exponentials for all samples
    exponentials = np.exp( Y_temp - c )
    
    # Get the value for the softmax function
    H = exponentials / np.sum(exponentials, axis = 0)
    return H


X = np.asarray([
    [0.16293629, 0.05701168, 0.45172072, 0.36527799, 0.0583312 ],
    [0.38338392, 0.84066432, 0.46417229, 0.46476983, 0.58996013],
    [0.97103108, 0.18383185, 0.77164735, 0.07827019, 0.84502797],
    [0.54848594, 0.35306614, 0.80574031, 0.43001758, 0.33883494],
    [0.15406714, 0.61009504, 0.92627007, 0.31464061, 0.96504647],
    [0.62869109, 0.96683923, 0.67307913, 0.76715599, 0.20064658],
    [0.13946135, 0.60860335, 0.12874316, 0.01669904, 0.56292157],
    [0.94717694, 0.95504354, 0.23629217, 0.89741572, 0.40860903],
    [0.45281581, 0.20396239, 0.96098094, 0.93523158, 0.05332813],
    [0.82914701, 0.54004522, 0.41405477, 0.42009503, 0.27472549],
])

theta = np.asarray([
    [0.83951097, 0.40424355, 0.31119862, 0.71417276, 0.12546929],
    [0.17836209, 0.44846427, 0.73829393, 0.48937756, 0.08848055],
])

temp_parameter = 0.39322872262244546

solution = np.asarray([
    [0.49780446, 0.59075737, 0.71055499, 0.57078338, 0.36703235, 0.66255384, 0.52216204, 0.85568887, 0.55831068, 0.759442],
    [0.50219554, 0.40924263, 0.28944501, 0.42921662, 0.63296765, 0.33744616, 0.47783796, 0.14431113, 0.44168932, 0.240558],
])

H = compute_probabilities(X, theta, temp_parameter)

print("Solution:", solution.shape)
print("H:", H.shape)

Solution: (2, 10)
H: (2, 10)


In [6]:
def compute_cost_function(X, Y, theta, lambda_factor, temp_parameter):
    """
    Computes the total cost over every datapoint.

    Args:
        X - (n, d) NumPy array (n datapoints each with d features)
        Y - (n, ) NumPy array containing the labels (a number from 0-9) for each
            data point
        theta - (k, d) NumPy array, where row j represents the parameters of our
                model for label j
        lambda_factor - the regularization constant (scalar)
        temp_parameter - the temperature parameter of softmax function (scalar)

    Returns
        c - the cost value (scalar)
    """
    k = theta.shape[0]
    n = X.shape[0]
    clip_prob_matrix = np.clip(compute_probabilities(X, theta, temp_parameter), 1e-15, 1-1e-15)
    log_clip_matrix = np.log(clip_prob_matrix)
    M = sparse.coo_matrix(([1]*n, (Y, range(n))), shape = (k,n)).toarray()
    error_term = (-1/n)*np.sum(log_clip_matrix[M==1])
    reg_term = (lambda_factor/2)*np.linalg.norm(theta)**2
    return error_term + reg_term

In [7]:
def run_gradient_descent_iteration(X, Y, theta, alpha, lambda_factor, temp_parameter):
    """
    Runs one step of batch gradient descent

    Args:
        X - (n, d) NumPy array (n datapoints each with d features)
        Y - (n, ) NumPy array containing the labels (a number from 0-9) for each
            data point
        theta - (k, d) NumPy array, where row j represents the parameters of our
                model for label j
        alpha - the learning rate (scalar)
        lambda_factor - the regularization constant (scalar)
        temp_parameter - the temperature parameter of softmax function (scalar)

    Returns:
        theta - (k, d) NumPy array that is the final value of parameters theta
    """
    itemp = 1./temp_parameter
    num_examples = X.shape[0]
    num_labels = theta.shape[0]
    probabilities = compute_probabilities(X, theta, temp_parameter)
    M = sparse.coo_matrix(([1]*num_examples, (Y,range(num_examples))), shape=(num_labels,num_examples)).toarray()
    non_regularized_gradient = np.dot(M-probabilities, X)
    non_regularized_gradient *= -itemp/num_examples
    return theta - alpha * (non_regularized_gradient + lambda_factor * theta)

In [17]:
from mnist.part1.softmax import softmax_regression, compute_test_error

temp_parameter = 1

theta, cost_function_history = softmax_regression(train_x, train_y, temp_parameter, alpha=0.3, lambda_factor=1.0e-4, k=10, num_iterations=150)
test_error = compute_test_error(test_x, test_y, theta, temp_parameter)

print("Test error:", test_error)

Test error: 0.10050000000000003


------

### 6. Changing Labels

#### Using the Current Model - Update Target

In [18]:
def update_y(train_y, test_y):
    """
    Changes the old digit labels for the training and test set for the new (mod 3)
    labels.

    Args:
        train_y - (n, ) NumPy array containing the labels (a number between 0-9)
                 for each datapoint in the training set
        test_y - (n, ) NumPy array containing the labels (a number between 0-9)
                for each datapoint in the test set

    Returns:
        train_y_mod3 - (n, ) NumPy array containing the new labels (a number between 0-2)
                     for each datapoint in the training set
        test_y_mod3 - (n, ) NumPy array containing the new labels (a number between 0-2)
                    for each datapoint in the test set
    """
    train_y_mod3 = np.mod(train_y,3)
    test_y_mod3 = np.mod(test_y,3)
    return train_y_mod3, test_y_mod3

In [20]:
def compute_test_error_mod3(X, Y, theta, temp_parameter):
    """
    Returns the error of these new labels when the classifier predicts the digit. (mod 3)

    Args:
        X - (n, d - 1) NumPy array (n datapoints each with d - 1 features)
        Y - (n, ) NumPy array containing the labels (a number from 0-2) for each
            data point
        theta - (k, d) NumPy array, where row j represents the parameters of our
                model for label j
        temp_parameter - the temperature parameter of softmax function (scalar)

    Returns:
        test_error - the error rate of the classifier (scalar)
    """
    from mnist.part1.softmax import get_classification
    predicted_label = get_classification(X,theta,temp_parameter)
    test_error = 1 - np.mean(np.mod(predicted_label,3)==Y)
    return test_error